In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current
client = MongoClient('mongodb://localhost:27017/')
result = client['project']['rental_info'].aggregate(
[
    {
        '$group': {
            '_id': '$cycle_num',
            'rental_count': {
                '$sum': 1
            }
        }
    }, {
        '$lookup': {
            'from': 'broken_history',
            'localField': '_id',
            'foreignField': 'cycle_num',
            'as': 'broken_info'
        }
    }, {
        '$addFields': {
            'broken_count': {
                '$size': '$broken_info'
            }
        }
    }, {
        '$match': {
            'rental_count': {
                '$ne': 0
            }
        }
    }, {
        '$addFields': {
            'broken_percentage': {
                '$multiply': [
                    {
                        '$divide': [
                            '$broken_count', '$rental_count'
                        ]
                    }, 100
                ]
            }
        }
    }, {
        '$project': {
            'cycle_num': '$_id',
            'rental_count': 1,
            'broken_count': 1,
            'broken_percentage': {
                '$round': [
                    '$broken_percentage', 2
                ]
            }
        }
    }, {
        '$sort': {
            'broken_percentage': -1,
            'rental_count': -1
        }
    }, {
        '$limit': 50
    }
]
)

# 결과를 DataFrame으로 변환
data = list(result)
df = pd.DataFrame(data)

# 막대차트 시각화
plt.figure(figsize=(12, 6))
plt.bar(df['cycle_num'], df['broken_percentage'], color='skyblue')
plt.xlabel('Cycle Number')
plt.ylabel('Broken Percentage')
plt.title('Broken Percentage by Cycle Number')
plt.xticks(rotation=90)
plt.tight_layout()

plt.show()

client.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pymongo import MongoClient

# MongoDB 연결 설정
client = MongoClient('mongodb://localhost:27017/')
results = list(client['project']['broken_history'].aggregate([
    {
        '$project': {
            'year': { '$year': '$created_at' },
            'month': { '$month': '$created_at' },
            'content': { '$trim': { 'input': '$content' } }  # content 필드의 좌우 공백 제거
        }
    }, {
        '$group': {
            '_id': { 'year': '$year', 'month': '$month' },
            'total_count': { '$sum': 1 },
            'details': { '$push': { 'content': '$content' } }
        }
    }, {
        '$unwind': '$details'
    }, {
        '$group': {
            '_id': {
                'year': '$_id.year',
                'month': '$_id.month',
                'content': '$details.content'
            },
            'count': { '$sum': 1 }
        }
    }, {
        '$group': {
            '_id': {
                'year': '$_id.year',
                'month': '$_id.month'
            },
            'total_count': { '$first': '$count' },
            'details': {
                '$push': {
                    'content': '$_id.content',
                    'count': '$count'
                }
            }
        }
    }, {
        '$project': {
            '_id': 0,
            'year': '$_id.year',
            'month': '$_id.month',
            'count': '$total_count',
            'detail': {
                '$sortArray': {
                    'input': '$details',
                    'sortBy': { 'count': -1 }
                }
            }
        }
    }, {
        '$sort': { 'year': 1, 'month': 1 }
    }
]))

# 결과가 비어 있는지 확인
if not results:
    print("No data found from MongoDB query.")
else:
    # content 값을 변환하기 위한 매핑
    content_mapping = {
        "기타": "other",
        "단말기": "terminal",
        "안장": "saddle",
        "체인": "chain",
        "타이어": "tire",
        "페달": "pedal"
    }

    # 데이터를 다중 막대 그래프 형식으로 변환
    data = []
    for result in results:
        year_month = f"{result['year']}-{result['month']:02d}"
        for detail in result['detail']:
            # content 값을 변환
            content_english = content_mapping.get(detail['content'], detail['content'])
            data.append([year_month, content_english, detail['count']])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=['Year-Month', 'Content', 'Count'])

    # 데이터 타입 변환
    df['Count'] = df['Count'].astype(int)

    # Year-Month를 문자열로 유지하고 오름차순으로 정렬
    df = df.sort_values(['Year-Month', 'Count'], ascending=[True, False])

    # 다중 막대 그래프를 위한 데이터 피벗
    pivot_df = df.pivot(index='Year-Month', columns='Content', values='Count').fillna(0)

    # 다중 막대 그래프 그리기
    pivot_df.plot(kind='bar', stacked=False, figsize=(10, 6))

    plt.title('Monthly Broken Data Report')
    plt.xlabel('Year-Month')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.legend(title='Content')
    plt.tight_layout()
    plt.show()